In [1]:
from matrix import Matrix, ZeroMatrix, OneMatrix
import numpy as np

In [17]:
class Recommender_UVD:
    # Assuming that "rating" is a numpy array
    def __init__(self, rating):
        self.rating = rating
        self.n_rows, self.n_cols = rating.shape
    
    def initialize_UV(self, d):
        self.d = d
        self.U = np.ones((self.n_rows, self.d))
        self.V = np.ones((self.d, self.n_cols))
    
    def U_rs(self, r, s):
        denom = 0
        nom = 0
        
        for j in range(self.n_cols):
            if self.rating[r-1][j] == 0:
                continue
                
            denom += self.V[s-1][j] ** 2
            
            temp = 0
            for k in range(self.d):
                temp += self.U[r-1][k] * self.V[k][j]
            nom += (self.rating[r-1][j] - temp) * self.V[s-1][j]
            
        result = nom / denom
        return result
    
    def V_rs(self, r, s): 
        denom = 0
        nom = 0
        
        for i in range(self.n_rows):
            if self.rating[i][s-1] == 0:
                continue
                
            denom += self.U[i][r-1] ** 2
            
            temp = 0
            for k in range(self.d):
                temp += self.U[i][k] * self.V[k][s-1]
            nom += (self.rating[i][s-1] - temp) * self.U[i][r-1]
            
        result = nom / denom
        return result
    
    def update_U(self, r, s, value):
        self.U[r-1][s-1] = value
        
    def update_V(self, r, s, value):
        self.V[r-1][s-1] = value
        
    def construct_M_hat(self):
        return self.U @ self.V
    
    def squared_Frobenius_norm(self):
        M_hat = self.construct_M_hat()
        
        sq_diff = 0
        
        for i in range(self.n_rows):
            for j in range(self.n_cols):
                sq_diff += (self.rating[i][j] - M_hat[i][j]) ** 2
                
        return sq_diff
    
    def sweep(self, sweep_count, d): # sweep_count = number of sweeps
        self.initialize_UV(d) # d- value (make OneMatrix for U and V)
        
        logs = []
        for a in range(sweep_count):
            # p is column num for U and row num for V
            for p in range(1,d+1): # start index from 1 because 1 is the first index on the matrix (1 is subtracted later)
                # q is row num for U and column num for V
                for q in range(1, self.n_rows+1): # 2 = d- value 
                    x = self.U_rs(q,p) # find value of Urs
                    self.update_U(q,p,x) # update U matrix with new value of Urs
                    #print(self.U)

                    y = self.V_rs(p,q) # find value of Vsr
                    self.update_V(p,q,y) # update V matrix with new value of Vsr
                    #print(self.V)

            logs.append(self.squared_Frobenius_norm)
        print(self.construct_M_hat())

        return logs



In [3]:
rating = np.array([[1,2,3],[0,4,5],[1,0,4]])
uvd = Recommender_UVD(rating)
uvd.sweep(2,2)

[[ 1.27164306  1.06764204  1.78126096]
 [ 5.2344284   2.65252884  3.39531134]
 [-0.39602034  1.13989097  2.77245097]]


38.97345196960585

In [38]:
rating = np.array([[1,2,3],[0,4,5],[1,0,4]])

uvd = Recommender_UVD(rating)
uvd.initialize_UV(2)
print(uvd.squared_Frobenius_norm())
print(uvd.rating)

x = uvd.U_rs(1,1)
print(x)
uvd.update_U(1,1,x)
print(uvd.squared_Frobenius_norm())
print(uvd.U)
y = uvd.construct_M_hat() # updated M_hat matrix 
print(y)

28.0
[[1 2 3]
 [0 4 5]
 [1 0 4]]
0.0
31.0
[[0. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1. 1.]
 [2. 2. 2.]
 [2. 2. 2.]]
